# package

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ─────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ───────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
library(sparklyr)


 次のパッケージを付け加えます: ‘sparklyr’ 


 以下のオブジェクトは ‘package:purrr’ からマスクされています:

    invoke


 以下のオブジェクトは ‘package:stats’ からマスクされています:

    filter




In [3]:
library(sparklyr.nested)

In [4]:
spark_conf <- sparklyr::spark_config()
spark_conf$spark.executor.cores <- 2
spark_conf$spark.executor.memory <- "8G"
spark_conf$spark.driver.memory <- "16G"

In [5]:
# spark_install()

In [6]:
## establish connection
sc <- sparklyr::spark_connect(master = "local")

# disease

In [7]:
diseasePath <- "./v23.02/diseases/"

In [8]:
## read dataset
disease <- sparklyr::spark_read_json(sc, path = diseasePath)

In [9]:
head(disease)

# Source: spark<?> [?? x 15]
  ancestors   children   code  dbXRefs descendants description directLocationIds
  <list>      <list>     <chr> <list>  <list>      <chr>       <list>           
1 <list [19]> <list [0]> http… <list>  <list [0]>  A mature T… <lgl [1]>        
2 <list [1]>  <list>     http… <list>  <list>      Genetic di… <lgl [1]>        
3 <list [16]> <list [4]> http… <list>  <list [4]>  A chronic … <lgl [1]>        
4 <list [4]>  <list [1]> http… <list>  <list [2]>  A disorder… <lgl [1]>        
5 <list [1]>  <list>     http… <list>  <list [77]> Anthropome… <lgl [1]>        
6 <list [4]>  <list [0]> http… <list>  <list [0]>  Atrophy of… <lgl [1]>        
# ℹ 8 more variables: id <chr>, indirectLocationIds <list>, name <chr>,
#   obsoleteTerms <list>, ontology <list>, parents <list>, synonyms <list>,
#   therapeuticAreas <list>

In [10]:
dim(disease)

[1] NA 15

## diseasetable

In [11]:
temp <- disease %>% 
    sparklyr::mutate(therapeuticAreasName = name) %>%
    sparklyr::select(id, therapeuticAreasName)

In [12]:
diseasetable <- disease %>% 
    sparklyr.nested::sdf_explode(therapeuticAreas)%>%
    sparklyr::left_join(temp, by = c("therapeuticAreas" = "id")) %>%
    sparklyr::select(therapeuticAreasName, id, name)

In [13]:
head(diseasetable)

# Source: spark<?> [?? x 3]
  therapeuticAreasName                    id          name                      
  <chr>                                   <chr>       <chr>                     
1 genetic, familial or congenital disease EFO_0000255 angioimmunoblastic T-cell…
2 cancer or benign tumor                  EFO_0000255 angioimmunoblastic T-cell…
3 hematologic disease                     EFO_0000255 angioimmunoblastic T-cell…
4 genetic, familial or congenital disease EFO_0000508 genetic disorder          
5 reproductive system or breast disease   EFO_0001054 leprosy                   
6 infectious disease                      EFO_0001054 leprosy                   

In [14]:
dim(diseasetable)

[1] NA  3

## diseasetable_ISD

In [ ]:
diseasetable_ISD <- diseasetable %>%
    sparklyr::filter(therapeuticAreasName == "immune system disease")

In [ ]:
head(diseasetable_ISD)

In [ ]:
dim(diseasetable_ISD)

# overall

In [15]:
overallPath <- "./v23.02/associationByOverallDirect/"

In [16]:
## read dataset
overall <- sparklyr::spark_read_json(sc, path = overallPath)

In [17]:
head(overall)

# Source: spark<?> [?? x 4]
  diseaseId     evidenceCount   score targetId       
  <chr>                 <dbl>   <dbl> <chr>          
1 MONDO_0004986             5 0.0104  ENSG00000000971
2 MONDO_0004986             2 0.00351 ENSG00000001626
3 MONDO_0004986             5 0.00317 ENSG00000002330
4 MONDO_0004986             1 0.00148 ENSG00000002587
5 MONDO_0004986             1 0.00148 ENSG00000002726
6 MONDO_0004986             2 0.00259 ENSG00000002745

In [18]:
dim(overall)

[1] NA  4

## overalltable

In [19]:
overalltable <- overall %>%
    sparklyr::mutate(overall = score) %>%
    sparklyr::select(diseaseId, targetId, overall) %>%
    sparklyr::right_join(diseasetable, by = c("diseaseId" = "id")) %>%
    sparklyr::select(therapeuticAreasName, diseaseId, name, targetId, overall) 

In [20]:
head(overalltable)

# Source: spark<?> [?? x 5]
  therapeuticAreasName                         diseaseId  name  targetId overall
  <chr>                                        <chr>      <chr> <chr>      <dbl>
1 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.0370
2 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.389 
3 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.0148
4 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.389 
5 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.268 
6 musculoskeletal or connective tissue disease EFO_00003… chil… ENSG000…  0.357 

In [21]:
dim(overalltable)

[1] NA  5

## overalltable_ISD

In [48]:
overalltable_ISD <- overalltable %>%
    sparklyr::filter(therapeuticAreasName == "immune system disease")

In [49]:
head(overalltable_ISD)

# Source: spark<?> [?? x 5]
  therapeuticAreasName  diseaseId   name                        targetId overall
  <chr>                 <chr>       <chr>                       <chr>      <dbl>
1 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.0370
2 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.389 
3 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.0148
4 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.389 
5 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.268 
6 immune system disease EFO_0000330 childhood acute myeloid le… ENSG000…  0.357 

In [50]:
dim(overalltable_ISD)

[1] NA  5

# datatype

In [ ]:
datatypePath <- "./v23.02/associationByDatatypeDirect/"

In [ ]:
## read dataset
datatype <- sparklyr::spark_read_json(sc, path = datatypePath)

In [ ]:
head(datatype)

In [ ]:
dim(datatype)

## datatypetable

In [ ]:
datatypetable <- datatype %>%
    sparklyr::select(diseaseId, targetId, datatypeId, score) %>%
    sparklyr::pivot_wider(names_from = datatypeId, values_from = score) %>%
    sparklyr::right_join(overalltable, by = c("diseaseId", "targetId")) %>%
    sparklyr::select(therapeuticAreasName, diseaseId, name, targetId, overall, 
                     affected_pathway, animal_model, genetic_association, known_drug, literature, rna_expression, somatic_mutation) 

In [ ]:
head(datatypetable)

In [ ]:
dim(datatypetable)

## datatypetable_ISD

In [ ]:
datatypetable_ISD <- datatypetable %>%
    sparklyr::filter(therapeuticAreasName == "immune system disease")

In [ ]:
head(datatypetable_ISD)

In [ ]:
dim(datatypetable_ISD)

# datasource

In [ ]:
datasourcePath <- "./v23.02/associationByDatasourceDirect/"

In [ ]:
## read dataset
datasource <- sparklyr::spark_read_json(sc, path = datasourcePath)

In [ ]:
head(datasource)

In [ ]:
dim(datasource)

## datasourcetable_somatic_mutation

In [ ]:
datasourcetable_somatic_mutation <- datasource %>%
    sparklyr::filter(datatypeId == "somatic_mutation") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_somatic_mutation)

In [ ]:
dim(datasourcetable_somatic_mutation)

## datasourcetable_genetic_association

In [ ]:
datasourcetable_genetic_association <- datasource %>%
    sparklyr::filter(datatypeId == "genetic_association") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_genetic_association)

In [ ]:
dim(datasourcetable_genetic_association)

## datasourcetable_affected_pathway

In [ ]:
datasourcetable_affected_pathway <- datasource %>%
    sparklyr::filter(datatypeId == "affected_pathway") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_affected_pathway)

In [ ]:
dim(datasourcetable_affected_pathway)

## datasourcetable_literature

In [ ]:
datasourcetable_literature <- datasource %>%
    sparklyr::filter(datatypeId == "literature") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_literature)

In [ ]:
dim(datasourcetable_literature)

## datasourcetable_rna_expression

In [ ]:
datasourcetable_rna_expression <- datasource %>%
    sparklyr::filter(datatypeId == "rna_expression") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_rna_expression)

In [ ]:
dim(datasourcetable_rna_expression)

## datasourcetable_animal_model

In [ ]:
datasourcetable_animal_model <- datasource %>%
    sparklyr::filter(datatypeId == "animal_model") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_animal_model)

In [ ]:
dim(datasourcetable_animal_model)

## datasourcetable_known_drug

In [ ]:
datasourcetable_known_drug <- datasource %>%
    sparklyr::filter(datatypeId == "known_drug") %>%
    sparklyr::select(diseaseId, targetId, datasourceId, score) %>%
    sparklyr::pivot_wider(names_from = datasourceId, values_from = score)

In [ ]:
head(datasourcetable_known_drug)

In [ ]:
dim(datasourcetable_known_drug)

# clusterProfiler

In [22]:
library(clusterProfiler)



clusterProfiler v4.6.2  For help: https://yulab-smu.top/biomedical-knowledge-mining-book/

If you use clusterProfiler in published research, please cite:
T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan, X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal enrichment tool for interpreting omics data. The Innovation. 2021, 2(3):100141


 次のパッケージを付け加えます: ‘clusterProfiler’ 


 以下のオブジェクトは ‘package:purrr’ からマスクされています:

    simplify


 以下のオブジェクトは ‘package:stats’ からマスクされています:

    filter




In [79]:
library(org.Hs.eg.db)

In [123]:
# library(mygene)

In [23]:
data(geneList, package="DOSE")

In [24]:
gene <- names(geneList)[abs(geneList) > 2]

In [25]:
# Entrez gene ID
head(gene)

[1] "4312"  "8318"  "10874" "55143" "55388" "991"

In [34]:
overall2 <- overall %>%
    sparklyr::collect()

In [81]:
# genetable <- mygene::queryMany(qterms = unique(overall2$targetId), 
#                                scopes = "ensembl.gene", 
#                                fields = "entrezgene", 
#                                species = "human")

In [85]:
# genetable2 <- genetable %>% 
#     as_tibble() %>%
#     dplyr::select(query, entrezgene) %>%
#     dplyr::filter(!is.na(entrezgene)) %>% 
#     dplyr::distinct_all()

In [82]:
keytypes(org.Hs.eg.db)

[1] "ACCNUM"       "ALIAS"        "ENSEMBL"      "ENSEMBLPROT"  "ENSEMBLTRANS"
 [6] "ENTREZID"     "ENZYME"       "EVIDENCE"     "EVIDENCEALL"  "GENENAME"    
[11] "GENETYPE"     "GO"           "GOALL"        "IPI"          "MAP"         
[16] "OMIM"         "ONTOLOGY"     "ONTOLOGYALL"  "PATH"         "PFAM"        
[21] "PMID"         "PROSITE"      "REFSEQ"       "SYMBOL"       "UCSCKG"      
[26] "UNIPROT"

In [83]:
genetable <- bitr(unique(overall2$targetId), 
                  fromType = "ENSEMBL", toType = "ENTREZID", 
                  OrgDb = "org.Hs.eg.db")

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(unique(overall2$targetId), fromType = "ENSEMBL", toType = "ENTREZID", :
“19.76% of input gene IDs are fail to map...”


In [84]:
head(genetable)

,ENSEMBL,ENTREZID
,<chr>,<chr>
1,ENSG00000000971,3075
2,ENSG00000001626,1080
3,ENSG00000002330,572
4,ENSG00000002587,9957
5,ENSG00000002726,26
6,ENSG00000002745,51384


In [88]:
genetable2 <- genetable %>% 
    as_tibble() %>%
    dplyr::filter(!is.na(ENTREZID)) %>% 
    dplyr::distinct_all() %>%
    dplyr::rename(query = ENSEMBL, entrezgene = ENTREZID)

In [51]:
overalltable_ISD2 <- overalltable_ISD %>%
    sparklyr::collect()

In [52]:
head(overalltable_ISD2)

therapeuticAreasName,diseaseId,name,targetId,overall
<chr>,<chr>,<chr>,<chr>,<dbl>
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000004468,0.03695799
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000014138,0.38930500
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000048052,0.01478319
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000062822,0.38930500
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000077092,0.26794539
immune system disease,EFO_0000330,childhood acute myeloid leukemia,ENSG00000077097,0.35674722


In [89]:
ID2GENE_ISD <- overalltable_ISD2 %>%
    dplyr::left_join(genetable2, by = join_by(targetId == query), relationship = "many-to-many") %>% 
    dplyr::select(diseaseId, entrezgene, overall) %>%
    dplyr::filter(!is.na(entrezgene)) %>%
    dplyr::rename(ID = diseaseId, GENE = entrezgene)

In [90]:
head(ID2GENE_ISD)

ID,GENE,overall
<chr>,<chr>,<dbl>
EFO_0000330,952,0.03695799
EFO_0000330,23649,0.38930500
EFO_0000330,9734,0.01478319
EFO_0000330,5424,0.38930500
EFO_0000330,5915,0.26794539
EFO_0000330,7155,0.35674722


In [91]:
ID2TERM_ISD <- overalltable_ISD2 %>%
    dplyr::select(diseaseId, name) %>%
    dplyr::distinct_all() %>% 
    dplyr::rename(ID = diseaseId, TERM = name)

In [92]:
head(ID2TERM_ISD)

ID,TERM
<chr>,<chr>
EFO_0000330,childhood acute myeloid leukemia
EFO_0000557,juvenile dermatomyositis
EFO_0000676,psoriasis
EFO_0002428,chronic myeloproliferative disorder
EFO_0002429,polycythemia vera
EFO_0002686,atopy


In [119]:
input_top <- 100

In [133]:
set_ID2GENE <- ID2GENE_ISD %>%
    dplyr::group_by(ID) %>% 
    dplyr::slice_max(n = input_top, order_by = overall) %>%
    dplyr::ungroup()

In [107]:
input_score_cutoff <- 0.01

In [108]:
set_ID2GENE <- ID2GENE_ISD %>%
    dplyr::filter(overall > input_score_cutoff)

In [124]:
result <- enricher(gene = gene, TERM2GENE = set_ID2GENE, TERM2NAME = ID2TERM_ISD,  
                   pvalueCutoff = 1, qvalueCutoff = 1, minGSSize = 1, maxGSSize = 20000)

In [134]:
head(set_ID2GENE)

ID,GENE,overall
<chr>,<chr>,<dbl>
EFO_0000094,10320,0.5613255
EFO_0000094,1029,0.4794530
EFO_0000094,2625,0.4650184
EFO_0000094,2078,0.4296292
EFO_0000094,930,0.4232721
EFO_0000094,916,0.4077874


In [125]:
result %>%
    setReadable(OrgDb = org.Hs.eg.db, keyType="ENTREZID") %>%
    as_tibble() %>%
    head()

ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
MONDO_0007918,"microcephaly with or without chorioretinopathy, lymphedema, or intellectual disability",4/99,20/6412,0.0002142699,0.07246564,0.06988624,PAX6/KIF11/BMP4/NDP,4
EFO_1000718,irritant dermatitis,5/99,42/6412,0.0004288051,0.07246564,0.06988624,S100A9/CXCL10/TPX2/CXCL11/CXCL9,5
EFO_0003834,cutaneous lupus erythematosus,7/99,100/6412,0.0008377531,0.07246564,0.06988624,MMP1/CXCL10/GZMB/LAG3/IDO1/COL17A1/SLC44A4,7
MONDO_0018982,Niemann-Pick disease type C,7/99,100/6412,0.0008377531,0.07246564,0.06988624,FOXM1/CXCL10/PBK/BIRC5/EZH2/IDO1/FOXA1,7
EFO_0008601,pemphigus foliaceus,6/99,100/6412,0.0043106341,0.18643493,0.17979882,MMP1/S100A9/FOXM1/EZH2/COMP/FOXA1,6
MONDO_0016168,cryopyrin-associated periodic syndrome,6/99,100/6412,0.0043106341,0.18643493,0.17979882,S100A9/S100A8/CXCL10/DLGAP5/CAMP/RARRES1,6


In [148]:
OT.set_ID2GENE <- function (input_top = NULL, input_score_cutoff = NULL) {
    if (!is.null(input_top) & is.null(input_score_cutoff)) {
        set_ID2GENE <- ID2GENE_ISD %>%
            dplyr::group_by(ID) %>% 
            dplyr::slice_max(n = input_top, order_by = overall) %>%
            dplyr::ungroup()
        
    } else if (is.null(input_top) & !is.null(input_score_cutoff)) {
        set_ID2GENE <- ID2GENE_ISD %>%
            dplyr::filter(overall > input_score_cutoff)
        
    } else if (!is.null(input_top) & !is.null(input_score_cutoff)) {
        set_ID2GENE <- ID2GENE_ISD
        
    } else {
        message("Error")
        set_ID2GENE <- NULL
    }
}

In [149]:
OT.enricher <- function (gene, set_ID2GENE, set_ID2TERM) {
    result <- enricher(gene = gene, TERM2GENE = prep.OT.enricher.out, TERM2NAME = ID2TERM_ISD,  
                      pvalueCutoff = 1, qvalueCutoff = 1, minGSSize = 1, maxGSSize = 20000)

    return(result)
}

In [152]:
set_ID2GENE.out <- OT.set_ID2GENE(input_top = 200)

In [153]:
result <- OT.enricher(gene = gene, set_ID2GENE = set_ID2GENE.out, set_ID2TERM = ID2TERM_ISD)

In [154]:
result %>%
    setReadable(OrgDb = org.Hs.eg.db, keyType="ENTREZID") %>%
    as_tibble() %>%
    head()

ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
EFO_0003834,cutaneous lupus erythematosus,15/127,200/8230,3.838195e-07,0.0001589013,0.0001527198,MMP1/S100A8/CXCL10/LAMP3/CXCL13/CXCL11/CXCL9/ADAMDEC1/GZMB/LAG3/IDO1/CCL8/GNLY/COL17A1/SLC44A4,15
MONDO_0018982,Niemann-Pick disease type C,14/127,200/8230,2.202821e-06,0.0004559839,0.0004382454,MMP1/FOXM1/CXCL10/RRM2/PBK/CXCL9/PAX6/BIRC5/EZH2/IDO1/AURKA/FOXA1/STC2/AGR2,14
MONDO_0007918,"microcephaly with or without chorioretinopathy, lymphedema, or intellectual disability",4/127,20/8230,2.163486e-04,0.0296354183,0.0284825531,PAX6/KIF11/BMP4/NDP,4
EFO_0008601,pemphigus foliaceus,11/127,203/8230,2.863325e-04,0.0296354183,0.0284825531,MMP1/S100A9/FOXM1/S100A8/CCL18/EZH2/MMP12/COMP/COL17A1/FABP4/FOXA1,11
EFO_1000718,irritant dermatitis,5/127,42/8230,4.353149e-04,0.0360440727,0.0346419006,S100A9/CXCL10/TPX2/CXCL11/CXCL9,5
EFO_1000668,allergic contact dermatitis,9/127,181/8230,1.885422e-03,0.1300941224,0.1250332526,MMP1/CXCL10/CXCL11/CXCL9/EZH2/GZMB/MMP12/CCL8/LRP2,9
